In [2]:
from azureml.train.hyperdrive import BayesianParameterSampling, choice, uniform
from azureml.train.hyperdrive import BanditPolicy
from azureml.core import Workspace
from azureml.core import Experiment, Environment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal

param_space = {
                 '--n_estimators': choice(5, 50, 100,500),
                 '--max_depth': choice(range(1,3))
              }

#Bayesian tries to select parameter combinations that will result in improved performance from the previous selection
param_sampling = BayesianParameterSampling(param_space)


#bandit policy to stop a run if the target performance metric underperforms the best run so far by a specified margin

early_termination_policy = BanditPolicy(slack_amount = 0.2,
                                        evaluation_interval=1,
                                        delay_evaluation=5)



hyperdrive = HyperDriveConfig(run_config=script_config,
                              hyperparameter_sampling=param_sampling,
                              policy=None,
                              primary_metric_name='Accuracy',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=6,
                              max_concurrent_runs=4)

experiment = Experiment(workspace = ws, name = 'hyperdrive_training')
hyperdrive_run = experiment.submit(config=hyperdrive)
